In [3]:
import tensorflow as tf
import numpy as np

# tf.contrib.rnn

* BasicRNNCell : The most basic RNN cell
* RNNCell : Abstract object representing an RNN cell
* BasicLSTMCell : Basic LSTM recurrent network cell
* LSTMCell : LSTM recurrent network cell
* GRUCell : Gated Recurrent Unit cell

# Construct cells 

In [6]:
hidden_size =50
num_layers = 3

In [7]:
cell = tf.contrib.rnn.GRUCell(hidden_size)

# Stack multiple cells 

* tf.nn.dynamic_rnn : uses a tf.While loop to dynamically construct the graph when it is executed. Graph creation is faster and you can feed batched of variable size
* tf.nn.bidirectional_dynamic_rnn : dynamic_rnn with bidirectional

In [8]:
rnn_cells = tf.contrib.rnn.MultiRNNCell([cell]*num_layers)

output, out_state = tf.nn.dynamic_rnn(cell, seq, <strong>length</strong>, initial_state) <br>
<br>
대부분의 시퀀스들은 길이가 다르다. 어떡할까

# Dealing with variable sequence length 

길이가 같아지도록 zero vecotor로 padding 해준다. <br>
대부분의 현존하는 모델들은 120 토큰보다 긴 시퀀스에는 잘 대처하지 못한다. 그래서 보통은 max_length를 고정해 놓고, 그것보다 길면 쳐낸다.

# Padded/truncated sequence length 

하지만 0으로 padded된 레이블은 total loss에 영향을 미치고 gradients에 영향을 주게 된다.

### Approach 1

* Maintain a mast(True for real, False for padded tokens) 진짜와 padding된 토큰을 구분해둔다
* Run your model on both the real/padded tokens(model will predict labels for the padded tokens as well)
* Only take into account the loss caused by the real elements

full_loss = tf.nn.softmax_cross_entropy_with_logits(preds, labels) <br>
loss = tf.reduce_mean(tf.boolean_mask(full_loss, mask))

실제 loss 계산할 때, 진짜 토큰만 고려한다

### Approach 2

* Let your model know the real sequence length so it only predict the labels for the real tokens

cell = tf.nn.rnn_cell.GRUCell(hidden_size) <br>
rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers) <br>
<strong>length = tf.reduce_sum(tf.reduce_max(tf.sign(seq), 2), 1)</strong> <br>
output, out_state = tf.nn.dynamic_rnn(cell, seq, <strong>length</strong>, initial_state)

# Vanishing Gradients 

### Use different activation units

* tf.nn.relu
* tf.nn.relu6
* tf.nn.crelu
* tf.nn.elu

### In addition to

* tf.nn.softplus
* tf.nn.softsign
* tf.nn.bias_add
* tf.sigmoid
* tf.tanh

# Exploding Gradients 

### Clip gradients with tf.clip_by_global_norm

gradients = tf.gradients(cost, tf.trainable_variables()) <br>
clipped\_gradients, _ = tf.clip_by_global_norm(gradients, max_grad_norm) <br>
optimizer = tf.train.AdamOptimizer(learning_rate) <br>
train_op = optimizer.apply_gradients(zip(gradients, trainables))

#### tf.gradients(ys, xs, grad_ys=None, name='gradients', colocate_gradients_with_ops=False, gate_gradients=False, aggregation_method=None)

Constructs symbolic partial derivatives of sum of ys w.r.t. x in xs.

ys and xs are each a Tensor or a list of tensors. grad_ys is a list of Tensor, holding the gradients received by the ys. The list must be the same length as ys.

(symbolic) Jacobian Matrix를 돌려주는건가

#### tf.clip_by_global_norm(t_list, clip_norm, use_norm=None, name=None)

Clips values of multiple tensors by the ratio of the sum of their norms.

To perform the clipping, the values t_list[i] are set to:

t_list[i] * clip_norm / max(global_norm, clip_norm)
where:

global_norm = sqrt(sum([l2norm(t)**2 for t in t_list])) <br><br>
If clip_norm > global_norm then the entries in t_list remain as they are, otherwise they're all shrunk by the global ratio.

# Anneal the learning rate(튜닝) 

Optimizers accept both scalars and tensors as learning rate

학습률에 step마다 decay를 가한다

learning_rate = tf.train.exponential_decay(init_lr,
 global_step,
 decay_steps,
 decay_rate,
 staircase=True) <br>
optimizer = tf.train.AdamOptimizer(learning_rate)

#### tf.train.exponential_decay(learning_rate, global_step, decay_steps, decay_rate, staircase=False, name=None)

Applies exponential decay to the learning rate.

decayed_learning_rate = learning_rate *
                        decay_rate ^ (global_step / decay_steps)


# Overfitting 

User dropout through tf.nn.dropout or DropoutWrapper for cells

#### tf.nn.dropout

hidden_layer = tf.nn.dropout(hidden_layer, keep_prob)

#### DropoutWrapper

cell = tf.nn.rnn_cell.GRUCell(hidden_size)<br>
cell = tf.nn.rnn_cell.DropoutWrapper(cell,
 output_keep_prob=keep_prob)